In [1]:
import os
import django
import requests
import datetime
import json
from tqdm import tqdm
import warnings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
warnings.filterwarnings(action='ignore')

In [2]:
import re
import requests
import pickle
from bs4 import BeautifulSoup as bs
from collections import Counter, OrderedDict

In [3]:
# https://developers.themoviedb.org/3/discover/movie-discover
URL = 'https://api.themoviedb.org/3'
API_KEY = '164acb58532a315bea423c96031d8a71'

## 장르

In [5]:
url_genres = f'{URL}/genre/movie/list?api_key=164acb58532a315bea423c96031d8a71'
result = requests.get(url_genres)
genres = result.json()['genres']

for g in genres:
    genre = Genre()
    genre.pk = g['id']
    genre.genre_name = g['name']
    genre.save()

## 영화(2000~2021)

In [6]:
for year in reversed(range(2000, 2022)):
    print(year)
    with open(f'./movie_{year}.json', 'r') as fp:
        movies = json.load(fp)

    # using bulk
    to_db_movies = []

    for mv in movies:
        movie = Movie()
        movie.pk = mv['id']
        movie.title_en = mv['original_title']
        movie.title_ko = mv['title']
        movie.rate = mv['vote_average']
        movie.rate_people_count = mv['vote_count']
        movie.poster_path = mv['poster_path']
        movie.description = mv['overview']
        movie.release_date = datetime.datetime.strptime(mv['release_date'], '%Y-%m-%d')
        to_db_movies.append(movie)

    temp = Movie.objects.bulk_create(to_db_movies)

    mgs = []
    for mv in movies:
        for gid in mv['genre_ids']:
            mg = MG()
            mg.movie_id = mv['id']
            mg.genre_id = gid
            mgs.append(mg)

    temp = MG.objects.bulk_create(mgs)

2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000


## 추천 관련 db

In [11]:
over2020 = Movie.objects.filter(release_date__gte='2000-01-01 00:00').order_by('-release_date')
movie_titles = [re.sub('[:.\-_ ]', '', x.title_ko) for x in over2020]
movie_pks = [x.pk for x in over2020]

In [10]:
def movie_order(kws):
    temp = []
    for k in kws:
        temp += k
    return list(OrderedDict(Counter(temp).most_common()).items())

In [7]:
with open('url_data_0528.pkl', 'rb') as f:
    url_data = pickle.load(f) # 단 한줄씩 읽어옴
with open('keyword_data_0528.pkl', 'rb') as f:
    keyword_data = pickle.load(f) # 단 한줄씩 읽어옴

### 추천 키워드

In [8]:
# keyword 저장
for key in keyword_data.keys():
    keyword = Keyword()
    keyword.word = key
    keyword.save()

### 키워드 DB 수집

In [12]:
for key in tqdm(keyword_data.keys()):

    word_pk = Keyword.objects.filter(word=key)[0].pk
    mv_od = movie_order(keyword_data[key])
    maximum = mv_od[0][1]
    minimum = min(2, 10 / maximum)

    for mo in mv_od:
        km = KM()
        idx = movie_titles.index(mo[0])
        movie_pk = Movie.objects.filter(pk=movie_pks[idx])[0].pk

        km.word_id = word_pk
        km.movie_id = movie_pk
        km.score = round(minimum + (10 - minimum) * (mo[1]-1)/maximum, 2)
        km.save()

100%|█████████████████████████████████████████████████████████████████████████████| 13/13 [01:34<00:00,  7.27s/it]
